In [1]:
import os
import pandas as pd
import glob
import numpy as np

In [2]:
os.getcwd()

'/net/vast-storage.ib.cluster/scratch/scratch/Tue/dclb/mmc/code/preprocessing/events_file_population'

In [3]:
twoback = pd.read_csv ("../../../sourcedata/behavioral/nback/twoback.txt", sep = '\t',header = None)

In [4]:
print(twoback)

     0   1  2
0   64  58  1
1  203  58  1
2  342  58  1


In [5]:
numrows_twoback = twoback.shape[0]
twoback[2]=['twoback' for val in range(0,numrows_twoback)]

In [6]:
print(twoback)

     0   1        2
0   64  58  twoback
1  203  58  twoback
2  342  58  twoback


In [7]:
zeroback = pd.read_csv ("../../../sourcedata/behavioral/nback/zeroback.txt", sep = '\t',header = None)

In [8]:
print(zeroback)

     0   1  2
0    2  58  1
1  141  58  1
2  280  58  1


In [9]:
numrows_zeroback = zeroback.shape[0]
zeroback[2]=['zeroback' for val in range(0,numrows_zeroback)]

In [10]:
print(zeroback)

     0   1         2
0    2  58  zeroback
1  141  58  zeroback
2  280  58  zeroback


In [11]:
events_df = pd.concat([zeroback,twoback], ignore_index=True)
events_df.columns = ['onset', 'duration', 'trial_type']
events_df

,onset,duration,trial_type
0,2,58,zeroback
1,141,58,zeroback
2,280,58,zeroback
3,64,58,twoback
4,203,58,twoback
5,342,58,twoback


In [12]:
events_df.sort_values(by='onset', inplace=True, ascending=True)
events_df

,onset,duration,trial_type
0,2,58,zeroback
3,64,58,twoback
1,141,58,zeroback
4,203,58,twoback
2,280,58,zeroback
5,342,58,twoback


In [13]:
nback_events_locations = (glob.glob("../../../sub-*/ses-*/func/*nback*events.tsv")) #this is the actual path, but troubleshooting with copies elsewhere first

#nback_events_locations = (glob.glob("../../*nback*events.tsv"))

In [14]:
print(nback_events_locations)

['../../../sub-HC006/ses-baseline/func/sub-HC006_ses-baseline_task-nback_rec-moco_run-1_events.tsv', '../../../sub-HC006/ses-baseline/func/sub-HC006_ses-baseline_task-nback_rec-unco_run-1_events.tsv', '../../../sub-HC007/ses-baseline/func/sub-HC007_ses-baseline_task-nback_rec-moco_run-1_events.tsv', '../../../sub-HC007/ses-baseline/func/sub-HC007_ses-baseline_task-nback_rec-unco_run-1_events.tsv', '../../../sub-HC008/ses-baseline/func/sub-HC008_ses-baseline_task-nback_rec-moco_run-1_events.tsv', '../../../sub-HC008/ses-baseline/func/sub-HC008_ses-baseline_task-nback_rec-unco_run-1_events.tsv', '../../../sub-HC009/ses-baseline/func/sub-HC009_ses-baseline_task-nback_rec-moco_run-1_events.tsv', '../../../sub-HC009/ses-baseline/func/sub-HC009_ses-baseline_task-nback_rec-unco_run-1_events.tsv', '../../../sub-HC010/ses-baseline/func/sub-HC010_ses-baseline_task-nback_rec-moco_run-1_events.tsv', '../../../sub-HC010/ses-baseline/func/sub-HC010_ses-baseline_task-nback_rec-unco_run-1_events.tsv',

In [15]:
#assign all events.tsv files the block times that we want to analyze
#set all other events.tsv files as empty files
#we want only the ones listed in the nifti_to_behavioral_run_match.tsv file since those are the ones corresponding to the nback run that was acquired last

#empty df 
empty_df = pd.DataFrame({'onset' : [],'duration' : [],'trial_type' : []})

#pull from nifti_to_behavioral_run_match.tsv
#make sure we don't read in the missing files (marked by 'none')
nifti_to_behavioral_run_match = pd.read_csv("nifti_to_behavioral_run_match.tsv", sep = '\t')
desired_nifti_files = list(nifti_to_behavioral_run_match['nifti_filename'])
desired_nifti_events_files = [file.split('bold')[0]+'events.tsv' for file in desired_nifti_files if file != 'none']

#for all nback events.tsv files, only add the stimulus timing if we are dealing with the correct run as listed in the nifti_to_behavioral_run_match.tsv file
#if nback events.tsv file but the run is not listed in the nifti_to_behavioral_run_match.tsv file then set events.tsv to empty tsv with only the correct headings
for file_path in nback_events_locations:
    file = file_path.split('/')[-1]
    if 'nback' in file:
        if file in desired_nifti_events_files:
            events_df.to_csv(file_path, index = False, header=True, sep='\t')
        else:
            empty_df.to_csv(file_path, index = False, header=True, sep='\t')
            

